In [1]:
# -*- coding: utf-8 -*-

from matplotlib import pyplot as plt
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import torch
import os
from torch.optim import *
from torch import nn
from torch.nn import functional as F
from scipy import ndimage
import pandas as pd
from copy import deepcopy
import random
from sklearn.metrics import f1_score
import torchvision.models as models


random.seed(7)
torch.cuda.manual_seed(7)
torch.manual_seed(7)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(7)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [2]:
DATA_ROOT = '/beegfs/bva212/openmic-2018'
OPENMIC = np.load(os.path.join(DATA_ROOT, 'openmic-2018.npz'))
X, Y_true, Y_mask, sample_key = OPENMIC['X'], OPENMIC['Y_true'], OPENMIC['Y_mask'], OPENMIC['sample_key']

In [3]:
train_samples = pd.read_csv(os.path.join(DATA_ROOT,'partitions/split01_train.csv'), names =['id']).to_numpy().squeeze()
test_samples = pd.read_csv(os.path.join(DATA_ROOT,'partitions/split01_test.csv'), names =['id']).to_numpy().squeeze()

In [4]:
len_data = len(train_samples)
idx_train = np.random.choice(len_data, int(len_data*0.8), replace=False)
remain_set = list(set(np.arange(len_data))-set(idx_train))

In [ ]:
val_idx = np.isin(sample_key, train_samples[remain_set])
train_idx = np.isin(sample_key, train_samples[idx_train])
test_idx = np.isin(sample_key, test_samples)

In [5]:
class ArrowOfTime(Dataset):

    def __init__(self, root_dir, files, weights, label):
        self.weights = weights
        self.device = device
        self.root_dir = root_dir
        self.files = files
        self.label = label

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        logscalogram = (np.load(self.root_dir + self.files[idx]+'_cqt.npy') - -24.3633)/14.2659
        len_cqt = logscalogram.shape[1]
        rem = len_cqt % 3
        logscalograms = np.split(logscalogram[:, :-rem], 3, axis = 1)
        logscalograms = np.stack(logscalograms)

        weight = self.weights[idx]
        label = (self.label[idx] > 0.5).astype(int)
        return {'logscalogram': logscalograms, 'label': label[np.newaxis, :], 'weight': weight[np.newaxis, :]}

filenames = []
root_dir = '/beegfs/bva212/openmic-2018/cqt_full/'

BATCH_SIZE = 32

def my_collate(batch):
    data = np.stack([item['logscalogram'] for item in batch])
    target = np.concatenate([item['label'] for item in batch],axis=0)
    weight = np.concatenate([item['weight'] for item in batch],axis=0)
    weight_sum = weight.sum(0)
    weight_sum = np.repeat(weight_sum[np.newaxis, :], len(batch), 0)
    weight_sum[weight_sum==0]=1
    weight = weight/weight_sum
    return [torch.from_numpy(data).float(), torch.from_numpy(target).float(), torch.from_numpy(weight).float()]


In [6]:
# Function for testing the model
def test_model(loader, model):
    correct = 0
    total_loss = 0
    total = 0
    total_num = 0
    actual_arr = []
    predicted_arr = []
    weight_array = []
    model.eval()
    with torch.no_grad():
        for spectrogram, target, weight in loader:
            spectrogram_batch, target_batch, weight_batch = spectrogram.to(device), target.to(device), weight.to(device)
            outputs = model(spectrogram_batch)
            loss = F.binary_cross_entropy_with_logits(outputs, target_batch,
                                                  weight = weight_batch,
                                                  reduction='sum')
            predicted = (torch.sigmoid(outputs.data)>0.5).float()
            
            actual_arr.extend(target.view(1,-1).squeeze().numpy().astype(int).tolist())
            predicted_arr.extend(predicted.view(1,-1).squeeze().cpu().numpy().astype(int).tolist())
            
            total_loss += loss.item()
            total += weight_batch.shape[0]
            
            correct += ((weight_batch != 0).float()*(predicted.eq(target_batch.view_as(predicted)).float())).sum().item()
            total_num += (weight_batch != 0).sum().item()
            weight_array = np.concatenate((weight_array,(weight != 0).reshape(-1).numpy().astype(int)))
        accuracy = (100 * correct / total_num)
        return accuracy, f1_score(actual_arr, predicted_arr, average='micro', sample_weight = weight_array), total_loss/total

def train_model(train_loader, val_loader, model, optimizer, scheduler, num_epochs):
    train_acc_list = []
    train_loss_list = []
    val_acc_list = []
    val_loss_list = []
    train_f1_list = []
    val_f1_list = []
    best_val_acc = 0
    for epoch in range(num_epochs):
        for spectrogram, target, weight in train_loader:
            model.train()
            spectrogram_batch, target_batch, weight_batch = spectrogram.to(device), target.to(device), weight.to(device)
            optimizer.zero_grad()
            outputs = model(spectrogram_batch)
            loss = F.binary_cross_entropy_with_logits(outputs, target_batch,
                                                  weight = weight_batch,
                                                  reduction='sum')
            loss.backward()
            optimizer.step()
        train_acc, f1_score_train, train_loss = test_model(train_loader, model)
        val_acc, f1_score_val, val_loss = test_model(val_loader, model)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state_dict = deepcopy(model.state_dict())
        
        train_acc_list.append(train_acc)
        train_f1_list.append(f1_score_train)
        train_loss_list.append(train_loss)
        val_f1_list.append(f1_score_val)
        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)
        
        scheduler.step(val_acc)
        print("Epoch:{}".format(epoch+1))
        print("Validation Accuracy:{:.2f}, Validation F1:{:.2f}, Val Loss: {:.5f}".format(val_acc, f1_score_val, val_loss))
        print("Training Acc: {:.2f}, Training F1:{:.2f}, Train Loss: {:.5f}".format(train_acc, f1_score_train, train_loss))
    return train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict

In [7]:
import pickle
sizes = [10, 50, 250, 500, 950, 'Full']

Model_Data = pickle.load(open('Training_Data_Index.pkl', 'rb'))

Val_dataset = ArrowOfTime(root_dir, sample_key[val_idx], Y_mask[val_idx], Y_true[val_idx])
Val_loader = torch.utils.data.DataLoader(dataset = Val_dataset, 
                                              batch_size = BATCH_SIZE,
                                              shuffle = False,
                                        collate_fn = my_collate)

results_dict = {}

for i in range(len(sizes)):
    if sizes[i] != 'Full':
        sample_key_train = Model_Data[i]
        idx_train_new = np.isin(sample_key, sample_key_train)
        Y_mask_train = Y_mask[idx_train_new]
        label_train = Y_true[idx_train_new]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train_new], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)
    else:
        Y_mask_train = Y_mask[train_idx]
        label_train = Y_true[train_idx]
        Train_dataset = ArrowOfTime(root_dir, sample_key[train_idx], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)

    
    weights = torch.load('/beegfs/sc6957/capstone/models/20191123/resnet_jigsaw_10_large_best_model.pth')['modelStateDict']
    reqd_wts = {key[7:]:val for key, val in weights.items()}
    model = models.resnet18(num_classes=2)
    model.load_state_dict(reqd_wts)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 20)
    model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    model = model.to(device)
    learning_rate = 10**(-5)
    num_epochs = 50 # number epoch to train

    optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=learning_rate, weight_decay = 0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True, threshold=0.03,
                                                           threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)

    train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict = train_model(Train_loader, Val_loader, model, optimizer, scheduler, num_epochs)
    results_dict[sizes[i]] = {
        'train_acc_list':train_acc_list,
        'train_loss_list': train_loss_list,
        'val_acc_list': val_acc_list,
        'val_loss_list': val_loss_list,
        'best_model_state_dict': best_model_state_dict
    }
pickle.dump(results_dict, open('/scratch/rc3620/Capstone/Size_Exp_resnet_Jigsaw_Wt_Results.pkl','wb'))

Epoch:1
Validation Accuracy:47.96, Validation F1:0.48, Val Loss: 0.44771
Training Acc: 46.62, Training F1:0.47, Train Loss: 0.47867
Epoch:2
Validation Accuracy:49.81, Validation F1:0.50, Val Loss: 0.43861
Training Acc: 49.10, Training F1:0.49, Train Loss: 0.44988
Epoch:3
Validation Accuracy:51.56, Validation F1:0.52, Val Loss: 0.43813
Training Acc: 51.58, Training F1:0.52, Train Loss: 0.44206
Epoch:4
Validation Accuracy:52.17, Validation F1:0.52, Val Loss: 0.43533
Training Acc: 54.95, Training F1:0.55, Train Loss: 0.43864
Epoch:5
Validation Accuracy:52.39, Validation F1:0.52, Val Loss: 0.43586
Training Acc: 59.91, Training F1:0.60, Train Loss: 0.41483
Epoch:6
Validation Accuracy:52.43, Validation F1:0.52, Val Loss: 0.44007
Training Acc: 65.32, Training F1:0.65, Train Loss: 0.38039
Epoch:7
Validation Accuracy:52.56, Validation F1:0.53, Val Loss: 0.43757
Training Acc: 71.17, Training F1:0.71, Train Loss: 0.36057
Epoch:8
Validation Accuracy:52.73, Validation F1:0.53, Val Loss: 0.43808
Tra

Epoch:12
Validation Accuracy:65.98, Validation F1:0.66, Val Loss: 0.36499
Training Acc: 97.10, Training F1:0.97, Train Loss: 0.18583
Epoch:13
Validation Accuracy:66.03, Validation F1:0.66, Val Loss: 0.36635
Training Acc: 97.63, Training F1:0.98, Train Loss: 0.17573
Epoch:14
Validation Accuracy:66.96, Validation F1:0.67, Val Loss: 0.36138
Training Acc: 98.16, Training F1:0.98, Train Loss: 0.15833
Epoch:15
Validation Accuracy:66.76, Validation F1:0.67, Val Loss: 0.35867
Training Acc: 98.70, Training F1:0.99, Train Loss: 0.14799
Epoch:16
Validation Accuracy:67.44, Validation F1:0.67, Val Loss: 0.35484
Training Acc: 98.94, Training F1:0.99, Train Loss: 0.13481
Epoch:17
Validation Accuracy:67.57, Validation F1:0.68, Val Loss: 0.35359
Training Acc: 99.23, Training F1:0.99, Train Loss: 0.12215
Epoch:18
Validation Accuracy:67.95, Validation F1:0.68, Val Loss: 0.35141
Training Acc: 99.47, Training F1:0.99, Train Loss: 0.10985
Epoch:19
Validation Accuracy:67.87, Validation F1:0.68, Val Loss: 0.3

Epoch:22
Validation Accuracy:78.82, Validation F1:0.79, Val Loss: 0.26303
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02672
Epoch:23
Validation Accuracy:79.03, Validation F1:0.79, Val Loss: 0.26985
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.02690
Epoch:24
Validation Accuracy:78.95, Validation F1:0.79, Val Loss: 0.27245
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02773
Epoch:25
Validation Accuracy:78.80, Validation F1:0.79, Val Loss: 0.27275
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02624
Epoch    25: reducing learning rate of group 0 to 1.0000e-07.
Epoch:26
Validation Accuracy:78.89, Validation F1:0.79, Val Loss: 0.26972
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02585
Epoch:27
Validation Accuracy:79.02, Validation F1:0.79, Val Loss: 0.27370
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02497
Epoch:28
Validation Accuracy:79.14, Validation F1:0.79, Val Loss: 0.26787
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.02439
E

Epoch:32
Validation Accuracy:83.89, Validation F1:0.84, Val Loss: 0.21963
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.01708
Epoch:33
Validation Accuracy:83.75, Validation F1:0.84, Val Loss: 0.21773
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.01699
Epoch:34
Validation Accuracy:84.01, Validation F1:0.84, Val Loss: 0.21289
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01703
Epoch:35
Validation Accuracy:83.94, Validation F1:0.84, Val Loss: 0.21912
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01713
Epoch:36
Validation Accuracy:83.92, Validation F1:0.84, Val Loss: 0.21518
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01652
Epoch:37
Validation Accuracy:83.92, Validation F1:0.84, Val Loss: 0.21552
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01682
Epoch:38
Validation Accuracy:83.85, Validation F1:0.84, Val Loss: 0.21850
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.01702
Epoch:39
Validation Accuracy:83.85, Validation F1:0.84, Val Loss

Epoch:42
Validation Accuracy:89.87, Validation F1:0.90, Val Loss: 0.15292
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.00549
Epoch:43
Validation Accuracy:89.84, Validation F1:0.90, Val Loss: 0.14854
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.00539
Epoch:44
Validation Accuracy:89.93, Validation F1:0.90, Val Loss: 0.15382
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.00548
Epoch:45
Validation Accuracy:89.83, Validation F1:0.90, Val Loss: 0.14761
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.00535
Epoch:46
Validation Accuracy:89.75, Validation F1:0.90, Val Loss: 0.15654
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.00537
Epoch:47
Validation Accuracy:89.81, Validation F1:0.90, Val Loss: 0.15483
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.00534
Epoch:48
Validation Accuracy:89.84, Validation F1:0.90, Val Loss: 0.14935
Training Acc: 100.00, Training F1:1.00, Train Loss: 0.00549
Epoch:49
Validation Accuracy:89.82, Validation F1:0.90, Val Lo

In [8]:
results_dict = {}
model1 = models.resnet18(num_classes=20).to(device)
model = models.resnet18(num_classes=20).to(device)
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
for i in range(len(sizes)):
    if sizes[i] != 'Full':
        sample_key_train = Model_Data[i]
        idx_train_new = np.isin(sample_key, sample_key_train)
        Y_mask_train = Y_mask[idx_train_new]
        label_train = Y_true[idx_train_new]
        Train_dataset = ArrowOfTime(root_dir, sample_key[idx_train_new], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)
    else:
        Y_mask_train = Y_mask[train_idx]
        label_train = Y_true[train_idx]
        Train_dataset = ArrowOfTime(root_dir, sample_key[train_idx], Y_mask_train, label_train)
        Train_loader = torch.utils.data.DataLoader(dataset = Train_dataset, 
                                                      batch_size = BATCH_SIZE,
                                                      shuffle = True,
                                                  collate_fn = my_collate)

    
    model.load_state_dict(model1.state_dict())
    learning_rate = 10**(-5)
    num_epochs = 50 # number epoch to train

    optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad == True], lr=learning_rate, weight_decay = 0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True, threshold=0.03,
                                                           threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)

    train_acc_list, train_loss_list, val_acc_list, val_loss_list, best_model_state_dict = train_model(Train_loader, Val_loader, model, optimizer, scheduler, num_epochs)
    results_dict[sizes[i]] = {
        'train_acc_list':train_acc_list,
        'train_loss_list': train_loss_list,
        'val_acc_list': val_acc_list,
        'val_loss_list': val_loss_list,
        'best_model_state_dict': best_model_state_dict
    }
pickle.dump(results_dict, open('/scratch/rc3620/Capstone/Size_Exp_Random_Wt_Resnet_Results.pkl','wb'))

Epoch:1
Validation Accuracy:50.40, Validation F1:0.50, Val Loss: 0.42198
Training Acc: 53.15, Training F1:0.53, Train Loss: 0.45469
Epoch:2
Validation Accuracy:50.59, Validation F1:0.51, Val Loss: 0.42273
Training Acc: 56.31, Training F1:0.56, Train Loss: 0.45277
Epoch:3
Validation Accuracy:51.94, Validation F1:0.52, Val Loss: 0.42340
Training Acc: 57.21, Training F1:0.57, Train Loss: 0.43500
Epoch:4
Validation Accuracy:52.73, Validation F1:0.53, Val Loss: 0.41881
Training Acc: 60.81, Training F1:0.61, Train Loss: 0.40633
Epoch:5
Validation Accuracy:53.46, Validation F1:0.53, Val Loss: 0.41948
Training Acc: 67.12, Training F1:0.67, Train Loss: 0.39452
Epoch:6
Validation Accuracy:54.37, Validation F1:0.54, Val Loss: 0.42500
Training Acc: 74.32, Training F1:0.74, Train Loss: 0.37350
Epoch:7
Validation Accuracy:54.84, Validation F1:0.55, Val Loss: 0.42309
Training Acc: 78.38, Training F1:0.78, Train Loss: 0.34594
Epoch:8
Validation Accuracy:55.52, Validation F1:0.56, Val Loss: 0.41957
Tra

Epoch:12
Validation Accuracy:69.07, Validation F1:0.69, Val Loss: 0.35755
Training Acc: 93.43, Training F1:0.93, Train Loss: 0.22487
Epoch:13
Validation Accuracy:69.48, Validation F1:0.69, Val Loss: 0.35373
Training Acc: 94.30, Training F1:0.94, Train Loss: 0.21534
Epoch:14
Validation Accuracy:69.60, Validation F1:0.70, Val Loss: 0.35122
Training Acc: 95.02, Training F1:0.95, Train Loss: 0.20304
Epoch:15
Validation Accuracy:70.17, Validation F1:0.70, Val Loss: 0.34932
Training Acc: 95.41, Training F1:0.95, Train Loss: 0.19096
Epoch:16
Validation Accuracy:70.73, Validation F1:0.71, Val Loss: 0.34639
Training Acc: 96.62, Training F1:0.97, Train Loss: 0.18116
Epoch:17
Validation Accuracy:70.58, Validation F1:0.71, Val Loss: 0.34500
Training Acc: 97.63, Training F1:0.98, Train Loss: 0.16938
Epoch:18
Validation Accuracy:70.68, Validation F1:0.71, Val Loss: 0.34060
Training Acc: 97.68, Training F1:0.98, Train Loss: 0.16004
Epoch:19
Validation Accuracy:70.90, Validation F1:0.71, Val Loss: 0.3

Epoch:22
Validation Accuracy:79.32, Validation F1:0.79, Val Loss: 0.25524
Training Acc: 99.82, Training F1:1.00, Train Loss: 0.05358
Epoch:23
Validation Accuracy:79.20, Validation F1:0.79, Val Loss: 0.26010
Training Acc: 99.82, Training F1:1.00, Train Loss: 0.05150
Epoch    23: reducing learning rate of group 0 to 1.0000e-07.
Epoch:24
Validation Accuracy:79.27, Validation F1:0.79, Val Loss: 0.26225
Training Acc: 99.85, Training F1:1.00, Train Loss: 0.05143
Epoch:25
Validation Accuracy:79.25, Validation F1:0.79, Val Loss: 0.25816
Training Acc: 99.88, Training F1:1.00, Train Loss: 0.04878
Epoch:26
Validation Accuracy:79.13, Validation F1:0.79, Val Loss: 0.26416
Training Acc: 99.88, Training F1:1.00, Train Loss: 0.04940
Epoch    26: reducing learning rate of group 0 to 1.0000e-08.
Epoch:27
Validation Accuracy:79.23, Validation F1:0.79, Val Loss: 0.26185
Training Acc: 99.84, Training F1:1.00, Train Loss: 0.05040
Epoch:28
Validation Accuracy:79.29, Validation F1:0.79, Val Loss: 0.25903
Trai

Epoch:32
Validation Accuracy:82.66, Validation F1:0.83, Val Loss: 0.22235
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.01941
Epoch:33
Validation Accuracy:83.10, Validation F1:0.83, Val Loss: 0.22326
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.01986
Epoch:34
Validation Accuracy:82.81, Validation F1:0.83, Val Loss: 0.22206
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.01938
Epoch:35
Validation Accuracy:82.83, Validation F1:0.83, Val Loss: 0.22375
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.01927
Epoch:36
Validation Accuracy:83.21, Validation F1:0.83, Val Loss: 0.22540
Training Acc: 99.98, Training F1:1.00, Train Loss: 0.01954
Epoch:37
Validation Accuracy:82.84, Validation F1:0.83, Val Loss: 0.22288
Training Acc: 99.98, Training F1:1.00, Train Loss: 0.01922
Epoch:38
Validation Accuracy:83.06, Validation F1:0.83, Val Loss: 0.22642
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.01955
Epoch:39
Validation Accuracy:82.72, Validation F1:0.83, Val Loss: 0.2

Epoch:43
Validation Accuracy:89.27, Validation F1:0.89, Val Loss: 0.15516
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.00581
Epoch:44
Validation Accuracy:89.38, Validation F1:0.89, Val Loss: 0.15644
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.00601
Epoch:45
Validation Accuracy:89.40, Validation F1:0.89, Val Loss: 0.15787
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.00605
Epoch:46
Validation Accuracy:89.36, Validation F1:0.89, Val Loss: 0.15797
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.00589
Epoch:47
Validation Accuracy:89.37, Validation F1:0.89, Val Loss: 0.16007
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.00579
Epoch:48
Validation Accuracy:89.44, Validation F1:0.89, Val Loss: 0.15618
Training Acc: 99.98, Training F1:1.00, Train Loss: 0.00616
Epoch:49
Validation Accuracy:89.26, Validation F1:0.89, Val Loss: 0.15605
Training Acc: 99.99, Training F1:1.00, Train Loss: 0.00566
Epoch:50
Validation Accuracy:89.36, Validation F1:0.89, Val Loss: 0.1

In [12]:
mean = 0.
std = 0.
for images, _, _ in Train_loader:
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images1 = images.reshape(batch_samples, images.size(1), -1)
    mean += images1.mean(2).sum(0)
    std += images1.std(2).sum(0)
    
mean /= len(Train_loader.dataset)
std /= len(Train_loader.dataset)

In [13]:
print(mean, std)

tensor([-24.3633]) tensor([14.2659])


In [65]:

model = conv_net().to(device)
model.load_state_dict(reqd_weights)

<All keys matched successfully>

In [9]:
pickle.dump(results_dict, open('Size_Exp_Jigsaw_Wt_Results.pkl'.format(sizes[i]),'wb'))

In [10]:
results_dict.keys()

dict_keys([10, 50, 250, 500])

In [8]:
import torchvision.models as models
resnet18 = models.resnet18().to(device)
alexnet = models.alexnet().to(device)
vgg16 = models.vgg16().to(device)
squeezenet = models.squeezenet1_0().to(device)
densenet = models.densenet161().to(device)
inception = models.inception_v3().to(device)


In [16]:
spectrogram, target, weight = next(iter(Train_loader))    
model.train()
spectrogram_batch, target_batch, weight_batch = spectrogram.to(device), target.to(device), weight.to(device)
x = model.conv1(spectrogram_batch)
x = model.bn1(x)
x = model.relu(x)
x = model.maxpool(x)

x = model.layer1(x)
x = model.layer2(x)
x = model.layer3(x)
x = model.layer4(x)

In [21]:
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
model.avgpool(x)

tensor([[[[1.4650]],

         [[0.5089]],

         [[0.9111]],

         ...,

         [[0.3773]],

         [[0.7808]],

         [[0.7331]]],


        [[[0.9524]],

         [[0.7230]],

         [[1.1426]],

         ...,

         [[1.1358]],

         [[1.3831]],

         [[1.1897]]],


        [[[1.9119]],

         [[1.6279]],

         [[0.7233]],

         ...,

         [[0.5244]],

         [[0.6860]],

         [[1.0922]]],


        ...,


        [[[0.4779]],

         [[0.7169]],

         [[1.0866]],

         ...,

         [[1.1316]],

         [[0.6493]],

         [[0.5833]]],


        [[[0.3052]],

         [[1.0795]],

         [[1.7370]],

         ...,

         [[0.6132]],

         [[2.3522]],

         [[1.9071]]],


        [[[0.1804]],

         [[0.4987]],

         [[1.0965]],

         ...,

         [[0.9759]],

         [[0.5855]],

         [[0.4740]]]], device='cuda:0', grad_fn=<ViewBackward>)